## Copy and Unzip Data and Labels

In [ ]:
# Python library for copying files from My Drive
import shutil

In [ ]:
# must be added for this notebook to work , and some to change the pre-processing arch.
#link: https://drive.google.com/file/d/1dR_-Nd6Ix0WOXxuJwoCq0eyZQDe9fruh/view?usp=sharing        this is test data
#link: https://drive.google.com/file/d/1jYhoOHa-808OVGdmh7wGIIFKnhPxyhg7/view?usp=sharing        this is ground truth
#link: https://drive.google.com/file/d/1IVpDDxPc8PghiUv0qU1SsJEywk6Ni0FL/view?usp=sharing        this is training
#link: https://drive.google.com/file/d/1ZX5nMQWM90oOouUskD_3QqvxgnTFxgW4/view?usp=sharing        this is bitwise training
#link: https://drive.google.com/file/d/1-XGsk5OgujjKMSNlfGSlOqAfB1GGt_xF/view?usp=sharing        this is bitwise test   
#link: https://drive.google.com/file/d/1-vron-Zc939CBHLI9I1UtV8WTodnBYW7/view?usp=sharing        this is limited crop training
#link: https://drive.google.com/file/d/1VM1cSZw0EtehUpc7xONTvbXnvyH9QqbV/view?usp=sharing        this is limited crop test

In [ ]:
# Ignore if running locally
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Ignore if running locally
# Otherwise change `drive/My Drive/Colab Notebooks/Data/ISIC2018_Task3_Training_*.zip` to /path/to/data
shutil.copy2("drive/My Drive/ISIC2018_Task3_Training_GroundTruth.zip", "./")#this is fixed
#shutil.copy2("/content/drive/My Drive/bitwise_realdata.zip", "/content/")# bitwise data or limited crop or normal
shutil.copy2("drive/My Drive/ISIC2018_Task3_Training_Input.zip", "./")#normal data

'./ISIC2018_Task3_Training_Input.zip'

In [ ]:
!unzip -q ISIC2018_Task3_Training_GroundTruth.zip

In [ ]:
!unzip -q ISIC2018_Task3_Training_Input.zip

In [ ]:
import os
import math
import pandas as pd
import numpy as np

In [ ]:
np.random.seed(42)

In [ ]:
df = pd.read_csv("ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv")

In [ ]:
for col in df.columns:
  if col != "image":
    s = df[col].sum()
    c = df[col].count()
    print("Column: {}\t Count: {}\t Percentage: {}".format(col, s, round((s/c)*100, 2)))

Column: MEL	 Count: 1113.0	 Percentage: 11.11
Column: NV	 Count: 6705.0	 Percentage: 66.95
Column: BCC	 Count: 514.0	 Percentage: 5.13
Column: AKIEC	 Count: 327.0	 Percentage: 3.27
Column: BKL	 Count: 1099.0	 Percentage: 10.97
Column: DF	 Count: 115.0	 Percentage: 1.15
Column: VASC	 Count: 142.0	 Percentage: 1.42


In [ ]:
# Image classes are in individual columns
# Created a new column name `class` 
# which contains values from 0-6
df["class"] = df.drop("image", axis=1).apply(lambda x: x.to_numpy().argmax(), axis=1)

In [ ]:
# What each value means in the `class` column
a = list(df.columns)
for i in ["image", "class"]:
  a.remove(i)
a = dict(zip(a, range(len(a))))
print(a)

{'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}


In [ ]:
df.drop(a.keys(), axis=1, inplace=True)

In [ ]:
df.to_csv("isic_labels.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2,random_state=42)

In [ ]:
train_df.to_csv("isic_train_labels.csv", index=False)
test_df.to_csv("isic_test_labels.csv", index=False)

In [ ]:
df = pd.read_csv("isic_train_labels.csv")

In [ ]:
# For data_block api to understand which image is for training
# and which one is for validation
train_df["is_valid"] = False
test_df["is_valid"] = True

In [ ]:
balanced = pd.concat([train_df, test_df]).reset_index(drop=True)

In [ ]:
print(balanced)

              image  class  is_valid
0      ISIC_0030888      1     False
1      ISIC_0031500      1     False
2      ISIC_0033752      0     False
3      ISIC_0024839      4     False
4      ISIC_0024438      1     False
...             ...    ...       ...
10010  ISIC_0026076      1      True
10011  ISIC_0026194      3      True
10012  ISIC_0026326      1      True
10013  ISIC_0025948      3      True
10014  ISIC_0029129      2      True

[10015 rows x 3 columns]


In [ ]:
# balanced[-10:]

In [ ]:
balanced.to_csv("isic_labels_balanced.csv", index=False)

## Start Coding FAST

In [ ]:
!curl https://course.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   322  100   322    0     0   1370      0 --:--:-- --:--:-- --:--:--  1370
Updating fastai...
Done.


In [ ]:
from fastai.vision import *

In [ ]:
#models.
"""
resnet101   
resnet152  
senet154 
vgg19_bn
densenet169    
vgg16_bn
densenet201 
 
"""

"""
chose if u want
"""

In [ ]:
# this is augmentation part by fastai api ,you can read its document to understand or shift space to find our used function in document
tfms = get_transforms(max_rotate=20, max_zoom=1.5, max_warp=0.4, p_affine=1., p_lighting=1., flip_vert=True)

In [ ]:
src = (ImageList.from_csv("./", "isic_labels_balanced.csv", folder="ISIC2018_Task3_Training_Input", suffix=".jpg")
                .split_from_df() # uses `is_valid` column
                .label_from_df())

After this, I did a bunch of training using various Resnet models and training sizes.

---


```
Recall(average="macro") == Balanced Accuracy
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [ ]:
from sklearn.utils import class_weight 
import numpy as np
#{'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
y_train=[3]*327+[2]*514+[4]*1099+[5]*115+[0]*1113+[1]*6705+[6]*142
class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
FL =nn.CrossEntropyLoss(weight=torch.cuda.FloatTensor(class_weights.astype(np.float32))) #nn.CrossEntropyLoss()

In [ ]:

#this code isn't optimized yes we repeated images into folders to find STD and mean but colab give you enough space
import os
os.makedirs("/content/directory", exist_ok=True)
 
os.makedirs("/content/directory", exist_ok=True)
os.makedirs("/content/directory/train", exist_ok=True)
os.makedirs("/content/directory/val", exist_ok=True)
os.makedirs("/content/directory/train/pos", exist_ok=True)
os.makedirs("/content/directory/val/pos", exist_ok=True)
os.makedirs("/content/directory/train/neg", exist_ok=True)
os.makedirs("/content/directory/val/neg", exist_ok=True)
 
for i, data in train_df.iterrows():
     if(data["class"]==0):
       shutil.copy2("/content/ISIC2018_Task3_Training_Input/"+data["image"]+".jpg", "/content/directory/train/pos")
     else:
       shutil.copy2("/content/ISIC2018_Task3_Training_Input/"+data["image"]+".jpg", "/content/directory/train/neg")
         
 
#for i, data in val_df.iterrows():
#     if(data["class"]==0):
#       shutil.copy2("/content/ISIC2018_Task3_Training_Input/"+data["image"]+".jpg", "/content/directory/val/pos")
#     else:
#       shutil.copy2("/content/ISIC2018_Task3_Training_Input/"+data["image"]+".jpg", "/content/directory/val/neg")

In [ ]:
# most important part most of fastai users can not find STD and mean easily so we used pytorch code
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
resize = transforms.Resize((224,224))
color_distort = transforms.ColorJitter(brightness=0.5) 
            # All transforms
 
 
data_transforms_new = {
    'train': transforms.Compose([
                    resize,
                    #transforms.RandomHorizontalFlip(),
                    #transforms.RandomVerticalFlip(),
                    #color_distort,
                    transforms.ToTensor()
                                  
        #transforms.Normalize((0.7106, 0.3751, 0.4817), (0.6935, 0.8864, 0.9258)),      
    ]),
    'val': transforms.Compose([
        resize,
        #color_distort,
        transforms.ToTensor(),
        #transforms.Normalize((0.7106, 0.3751, 0.4817), (0.6935, 0.8864, 0.9258)),
       
        
    ])
}
 
image_datasets_={"train":datasets.ImageFolder("/content/directory/train",data_transforms_new["train"])}
 
loader = torch.utils.data.DataLoader(
    image_datasets_["train"],
    batch_size=16,
    num_workers=4,
    shuffle=False)
 
 
def online_mean_and_sd(loader):
    """Compute the mean and sd in an online fashion
 
        Var[x] = E[X^2] - E^2[X]
    """
    cnt = 0
    fst_moment = torch.empty(3)
    snd_moment = torch.empty(3)
    #for  (images,x) in (loader):
    #  print(images)
    for images,_ in loader:
 
        b, c, h, w = images.shape
        nb_pixels = b * h * w
        sum_ = torch.sum(images, dim=[0, 2, 3])
        sum_of_square = torch.sum(images ** 2, dim=[0, 2, 3])
        fst_moment = (cnt * fst_moment + sum_) / (cnt + nb_pixels)
        snd_moment = (cnt * snd_moment + sum_of_square) / (cnt + nb_pixels)
 
        cnt += nb_pixels
 
    return fst_moment, torch.sqrt(snd_moment - fst_moment ** 2)
std=online_mean_and_sd(loader)
print(std)#mean not zero and std is not 1, this will give u mean and std

(tensor([0.7632, 0.5467, 0.5711]), tensor([0.1404, 0.1518, 0.1691]))


In [ ]:
my_norm=([0.7632, 0.5467, 0.5711], [0.1404, 0.1518, 0.1691])
data = (src.transform(tfms, size=224)
        .databunch().normalize(my_norm))
data.batch_size=32

In [ ]:
import shutil
shutil.copy2("/content/drive/My Drive/ISIC2018_Task3_Test_Input.zip","/content/")

'/content/ISIC2018_Task3_Test_Input.zip'

In [ ]:
!unzip -q ISIC2018_Task3_Test_Input.zip                   

In [ ]:
#model_name="senet154_non_seg"
#test_df.to_csv("/content/drive/My Drive/isic_test_labels_"+model_name+".csv", index=False)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
SaveBestModel.names ="dens201"                          #models
learn = cnn_learner(data, models.densenet201, metrics=[Recall(average="macro"),Precision(average="macro")], loss_func=FL)

In [ ]:
epochs=55
learn.fit_one_cycle(epochs)

In [ ]:
x=learn.validate()

In [ ]:
with open("/content/submission_"+"dens161"+"_ep"+str("no")+".csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image","MEL","NV","BCC","AKIEC","BKL","DF","VASC"])
    for i in range(len(li_re)):
        writer.writerow(li_re[i])

In [ ]:
cv=pd.read_csv("/content/submission_dens161_epno.csv")
cv

In [ ]:
cs=["",0.0,0.0,0.0,0.0,0.0,0.0,0.0]
li_re=[]
import glob
import csv
import glob
counter=0
#{'MEL': 0, 'NV': 1, 'BCC': 2, 'AKIEC': 3, 'BKL': 4, 'DF': 5, 'VASC': 6}
##['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
for filename in glob.glob("/content/ISIC2018_Task3_Test_Input/*.jpg"): #assuming gif 
    path = filename.split("/")
    name = path[len(path)-1]  
    parts=name.split(".")
    ids=parts[0]
    copy=list(cs)
    copy[0]=ids
    img = open_image(filename)
    #img = img.apply_tfms(tfms=get_transforms()[1], size=224, resize_method=3)
    x,y,z=learn.predict(img)
    prob = F.softmax(Variable(torch.Tensor(np.array(z))), dim=0)
    arr1=np.array(prob) 
    arr=[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
    arr[4]=arr1[3]
    arr[3]=arr1[2]
    arr[5]=arr1[4]
    arr[6]=arr1[5]
    arr[1]=arr1[0]
    arr[2]=arr1[1]
    arr[7]=arr1[6]
    mm=arr.index(max(arr))
    for i in range(len(arr)):
      arr[i]=0
    arr[mm]=1.0
    copy[4]=arr[4]
    copy[3]=arr[3]
    copy[5]=arr[5]
    copy[6]=arr[6]
    copy[1]=arr[1]
    copy[2]=arr[2]
    copy[7]=arr[7]
 
    #copy[4]=arr1[3]
    #copy[3]=arr1[2]
    #copy[5]=arr1[4]
    #copy[6]=arr1[5]
    #copy[1]=arr1[0]
    #copy[2]=arr1[1]
    #copy[7]=arr1[6]
 
    li_re.append(copy)
    counter+=1
    if(counter%100==0):
      print(counter)

In [ ]:
with open("/content/drive/My Drive/submission_"+model_name+"_ep"+str(epochs)+".csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["image","MEL","NV","BCC","AKIEC","BKL","DF","VASC"])
    for i in range(len(li_re)):
        writer.writerow(li_re[i])

In [ ]:
learn.recorder.plot_losses()
plt.yscale('log')

In [ ]:
learn.export()
shutil.move("/content/export.pkl", "/content/drive/My Drive/_"+model_name+"_exported_"+str(epochs)+".pkl")